In [10]:
import torch
import kagglehub
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import get_scheduler, PreTrainedTokenizerFast
from tqdm.auto import tqdm
import evaluate
from accelerate import Accelerator
from models import NextByteTransformer
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from recipe_nlg import TokenizedRecipeNLGDataset

"""Model & training hyper parameters"""
context_length = 512
d_model = 512
num_heads = 8
num_hidden_layers = 8
d_hidden = 2048
num_decoders = 1
num_epochs = 5
lr = 3e-5
batch_size = 16
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)


# different loss function?
loss_fn = nn.CrossEntropyLoss()

# set mode and tokenizer path
mode = 'title_to_all'
tokenizer_path = Path('Tokenizers/' + mode + '_tokenizer')

print('loading tokenizer')
tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path, model_max_length=context_length)


cuda
loading tokenizer


In [11]:
print('loading df..')
path = kagglehub.dataset_download("paultimothymooney/recipenlg")
# Load the dataset
df = pd.read_csv(path + "/RecipeNLG_dataset.csv", header=0)
print(len(df))


loading df..
2231142


In [12]:
# df = df[:10000]
# print(len(df))

In [13]:
print('splitting into train and test sets')
train_df, eval_df = train_test_split(df, test_size=0.2)

print('creating datasets..')
train_dataset = TokenizedRecipeNLGDataset(df=train_df, tokenizer=tokenizer, mode='all')
eval_dataset = TokenizedRecipeNLGDataset(df=eval_df, tokenizer=tokenizer, mode='all')

splitting into train and test sets
creating datasets..
RECIPE DEVICE: cuda
RECIPE DEVICE: cuda


In [14]:
print('creating model..')
# declare model
model = NextByteTransformer(
    vocab_size=20000,
    context_length=context_length,
    d_model=d_model,
    num_heads=num_heads,
    num_hidden_layers=num_hidden_layers,
    d_hidden=d_hidden,
    num_decoders=num_decoders
)

creating model..


In [15]:
print('creating dataloaders')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=True)

# check shape
for batch in train_dataloader:
    print(batch['input_ids'].shape)
    break

creating dataloaders
torch.Size([16, 511])


In [16]:
# # TODO: explain what this is
optimizer = AdamW(model.parameters(), lr=lr)

model.to(device)

# # TODO: explain what this does
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [17]:
model.train()
for epoch in range(num_epochs):
    print(f"EPOCH {epoch}")
    for batch in tqdm(train_dataloader, unit='batch'):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        logits = model(input_ids)
        
        # Reshape logits and labels to match cross-entropy expectations
        logits = logits.view(-1, logits.size(-1))  # (batch_size * seq_len, vocab_size)
        labels = labels.view(-1)  # (batch_size * seq_len)

        # Compute loss
        loss = loss_fn(logits, labels)
        
        # Update model
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

  

EPOCH 0


  0%|          | 0/111558 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [ ]:
# Put model in eval mode
model.eval()

# Encode the prompt
input_ids = tokenizer.encode("burger", return_tensors="pt").to(device)

# Number of tokens to generate
max_new_tokens = 100

# Autoregressive generation
with torch.no_grad():
    for _ in range(max_new_tokens):
        logits = model(input_ids)
        next_token_logits = logits[:, -1, :]

        # Pick next token (argmax or sample)
        next_token_id = next_token_logits.argmax(dim=-1, keepdim=True)

        # Append to input
        input_ids = torch.cat([input_ids, next_token_id], dim=-1)

# Decode full sequence
generated_text = tokenizer.decode(input_ids.squeeze().tolist())
print(generated_text)